In [176]:
# RSA simple implementing
# 
# 质数
#   只能被1和自身整除
#
# 互质
#   最大公约数为1的数
#
# 欧拉函数
#   φ(pq) = φ(p)φ(q) = (p - 1)(q - 1)
#   r = φ(pq) = lcm(p - 1, q - 1)
#
# 模反元素
#   如果两个正整数a和n互质，那么一定可以找到整数b，使得 ab-1 被n整除，或者说ab被n除的余数是1
#   ab ≡ 1(mod n)  
#   ed ≡ 1(mod r)
#
# 欧拉定理
#   如果两个正整数a和n互质，则n的欧拉函数 φ(n) 可以让下面的等式成立：
#   a^φ(n) ≡ 1(mod n)
#   a * a^(φ(n) - 1) ≡ 1(mod n)
#   由此可得：a的φ(n) - 1次方肯定是a关于n的模反元素
#   欧拉定理就可以用来证明模反元素必然存在
#
# 费马小定理
#   假设正整数a与质数p互质，因为质数p的φ(p)等于p-1，则欧拉定理可以写成
#   a^(p-1) ≡ 1 (mod p)
#   费马小定理是欧拉定理的一个特例
#
#
# RSA generator steps
# step1: p, q : two different primes
#   n = p * q  : modulus
# step2:
#   φ(pq) = φ(p)φ(q) = (p - 1) * (q - 1)
#   r = φ(pq) = lcm(p - 1, q - 1)
# step3: 
#   e * d ≡ 1(mod r) -> e * d % r = 1 
#   e: 1 < e < r and gdc(e, r) == 1
# step4:
#   d: 1 < d < r and d * e mod r == 1
# step5:
#   (e, n) : public key
#   (d, n) : private key
#
# RSA encrypt and decrypt
#   m: message, m_e: encrypted message, m_e_d: decrypted message from m_e
# 
#   m_e ≡ m^e (mod n)
#   m_e_d ≡ m_e^d (mod n) ≡ m^(ed) (mod n)
#   ed ≡ 1 (mod φ(n)) -> ed = hφ(n) + 1
#   m_e_d ≡ m^(hφ(n) + 1) (mod n) ≡ m * m^(hφ(n)) ≡ m * (m^φ(n))^h ≡ m * (1)^h (mod n) ≡ m
# 
# reference:
#   https://zh.wikipedia.org/wiki/RSA%E5%8A%A0%E5%AF%86%E6%BC%94%E7%AE%97%E6%B3%95
#   https://en.wikipedia.org/wiki/RSA_(cryptosystem)
#   https://zhuanlan.zhihu.com/p/33580225
#

In [237]:
IGNORE_FIRST_N_PRIMES = 5
P_AND_Q_DVALUE_THRESHOLD = 5
E_AND_D_DVALUE_THRESHOLD = 5

In [238]:
def primes_generator():
    primes = []   # primes generated so far
    last = 1      # last number tried
    while True:
        last += 1
        for p in primes:
            if last % p == 0:
                break
        else:
            primes.append(last)
            yield last

# greatest common divisor
def gcd(x, y):
   while(y):
       x, y = y, x % y
        
   return x

# least common multiple
def lcm(x, y):
   return (x * y) // gcd(x, y)

In [239]:
def get_p_q():
    primes = primes_generator()
    for i in range(1, IGNORE_FIRST_N_PRIMES):
        next(primes)   
    p = next(primes)

    for i in range(1, P_AND_Q_DVALUE_THRESHOLD):
        next(primes)
    q = next(primes)
    
    return (p, q)
        
def get_n(p, q):
    return p * q


def get_r(p, q):
    return lcm(p - 1, q - 1)


def get_e(r):
    for e in range(2, r):
        if gcd(e, r) == 1:
            return e
        else:
            continue
            
    return 0

    
def get_d(r, e):
    for d in range(2, r):
        if d == e and abs(d - e) < E_AND_D_DVALUE_THRESHOLD:
            continue
            
        if e * d % r == 1:
            return d
        else:
            continue
            
    return 0

In [240]:
def get_rsa(p, q):
    rsa = dict()
    rsa['p'] = p
    rsa['q'] = q
    rsa['n'] = p * q
    
    rsa['r'] = get_r(p, q)
    r = rsa['r']
    e = get_e(r)
    if e == 0:
        rsa['e'] = 0
        rsa['ready'] = False
        return rsa
    
    d = get_d(r, e)
    if d == 0:
        rsa['d'] = 0        
        rsa['ready'] = False
        return rsa
    
    rsa['e'] = e
    rsa['d'] = d
    rsa['ready'] = True
    
    return rsa

In [277]:
def verify_rsa(rsa, msgs):
    for e, d in (('e', 'd'), ('d', 'e')):
        print("Pass: m -> {}(m) -> {}({}(m)) -> m ...".format(e, d, e))
        for m in msgs:
            m_e = m ** rsa[e] % rsa['n']
            m_e_d = m_e ** rsa[d] % rsa['n']
            print("{status}: {m:4d} -> {m_e:4d} -> {m_e_d:4d}".format(
                status=(m == m_e_d), 
                m=m, 
                m_e=m_e, 
                m_e_d=m_e_d)
            )
        print("")

In [278]:
def main():
    p, q = get_p_q()
    rsa = get_rsa(p, q)
    print('RSA -> {}'.format(rsa))
    if rsa['ready'] == False:
        print('Get RSA failed!')
        return
    
    CASE_COUNT = 20
    print('RSA verify:')
    print("msg < rsa['n'] cases:")
    verify_rsa(rsa, range(max(rsa['n'] - CASE_COUNT, 1), rsa['n']))
    print("msg >= rsa['n'] cases:")
    verify_rsa(rsa, range(rsa['n'], rsa['n'] + CASE_COUNT))

In [279]:
main()

RSA -> {'p': 11, 'q': 29, 'n': 319, 'r': 140, 'e': 3, 'd': 47, 'ready': True}
RSA verify:
msg < rsa['n'] cases:
Pass: m -> e(m) -> d(e(m)) -> m ...
True:  299 ->  294 ->  299
True:  300 ->  159 ->  300
True:  301 ->  229 ->  301
True:  302 ->  191 ->  302
True:  303 ->   51 ->  303
True:  304 ->  134 ->  304
True:  305 ->  127 ->  305
True:  306 ->   36 ->  306
True:  307 ->  186 ->  307
True:  308 ->  264 ->  308
True:  309 ->  276 ->  309
True:  310 ->  228 ->  310
True:  311 ->  126 ->  311
True:  312 ->  295 ->  312
True:  313 ->  103 ->  313
True:  314 ->  194 ->  314
True:  315 ->  255 ->  315
True:  316 ->  292 ->  316
True:  317 ->  311 ->  317
True:  318 ->  318 ->  318

Pass: m -> d(m) -> e(d(m)) -> m ...
True:  299 ->  150 ->  299
True:  300 ->  108 ->  300
True:  301 ->  247 ->  301
True:  302 ->  278 ->  302
True:  303 ->  151 ->  303
True:  304 ->   39 ->  304
True:  305 ->  222 ->  305
True:  306 ->   81 ->  306
True:  307 ->  186 ->  307
True:  308 ->  275 ->  308
True: